In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np
import tinygrad.optim as optim
from extra.utils import get_parameters


%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
seedmat = np.random.randn(3,3).astype(np.float32)

In [3]:
x_init = np.random.randn(3,3).astype(np.float32)
x = DenseTensor(x_init)
y = DenseTensor(np.random.randn(3,3))

warning, (3, 3) isn't float32, it's float64


In [4]:
class MLP:
  def __init__(self):
    self.W = DenseTensor(seedmat)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    #out = out.logsoftmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [5]:
class MLP2:
  def __init__(self):
    self.W = SparseTensor(seedmat)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    #out = out.logsoftmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [6]:
def loss_fn(y, y_pred):
    return ((y-y_pred)**2)**.5

### Dense

In [7]:
model = MLP()

In [8]:
optimizer = optim.SGD(model.parameters(), lr=.0001)

In [9]:
optimizer.zero_grad()
res = model.forward(x)
loss = loss_fn(res, y)
loss.backward()
#optimizer.step()

\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (1,)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (1,)> with grad None>
T/G: <DenseTensor <GPUBuffer with shape (3, 3)> with grad None> <GPUBuffer with shape (3, 3)> (3, 3) (3, 3)
T/G: <DenseTensor <GPUBuffer with shape (1,)> with grad None> <GPUBuffer with shape (1,)> (1,) (1,)
T/G: <DenseTensor <GPUBuffer with shape (3, 3)> with grad None> <GPUBuffer with shape (3, 3)> (3, 3) (3, 3)
T/G: <DenseTensor <GPUBuffer with shape (1,)> with grad None

In [10]:
model.parameters()

[<DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>,
 <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>]

In [11]:
loss.cpu().data

array([[0.50551486, 0.8109138 , 0.48307404],
       [2.6015759 , 1.4273951 , 2.0545573 ],
       [0.4443901 , 0.83480704, 2.1205204 ]], dtype=float32)

In [12]:
res.cpu().data

array([[-0.6899253 ,  0.22909799,  0.17805669],
       [-2.8184183 , -0.08430927,  1.1420828 ],
       [ 0.2419483 ,  0.05442254, -0.11403634]], dtype=float32)

In [13]:
res.grad.cpu().data

array([[ 0.99999994,  1.        ,  1.        ],
       [-1.        ,  0.99999994,  0.99999994],
       [ 0.9999999 ,  1.        , -1.        ]], dtype=float32)

In [14]:
model.W.grad.cpu().data

array([[-0.2912428 ,  0.7490896 , -0.577783  ],
       [-0.98378825,  0.35937876,  0.76732665],
       [ 1.5149807 , -1.7959167 , -1.8263358 ]], dtype=float32)

### Second

In [15]:
model2 = MLP2()

In [16]:
optimizer2 = optim.SGD(model.parameters(), lr=.0001)

In [17]:
optimizer2.zero_grad()
res2 = model.forward(x)
loss2 = loss_fn(res2, y)
loss2.backward()
#optimizer2.step()

\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>
\nVISIT <DenseTensor <GPUBuffer with shape (1,)> with grad None>
\nVISIT <DenseTensor <GPUBuffer with shape (1,)> with grad None>
T/G: <DenseTensor <GPUBuffer with shape (3, 3)> with grad None> <GPUBuffer with shape (3, 3)> (3, 3) (3, 3)
T/G: <DenseTensor <GPUBuffer with shape (1,)> with grad None> <GPUBuffer with shape (1,)> (1,) (1,)
T/G: <DenseTensor <GPUBuffer with shape (3, 3)> with grad None> <GPUBuffer with shape (3, 3)> (3, 3) (3, 3)
T/G: <Dens

In [18]:
res2.cpu().data==res.cpu().data

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [19]:
res2.cpu().data

array([[-0.6899253 ,  0.22909799,  0.17805669],
       [-2.8184183 , -0.08430927,  1.1420828 ],
       [ 0.2419483 ,  0.05442254, -0.11403634]], dtype=float32)

In [20]:
loss.cpu().data==loss2.cpu().data

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [21]:
loss2.cpu().data

array([[0.50551486, 0.8109138 , 0.48307404],
       [2.6015759 , 1.4273951 , 2.0545573 ],
       [0.4443901 , 0.83480704, 2.1205204 ]], dtype=float32)

In [22]:
res.grad.cpu().data==res2.grad.cpu().data

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [23]:
res2.grad.cpu().data

array([[ 0.99999994,  1.        ,  1.        ],
       [-1.        ,  0.99999994,  0.99999994],
       [ 0.9999999 ,  1.        , -1.        ]], dtype=float32)

In [24]:
model.W, model2.W

(<DenseTensor <GPUBuffer with shape (3, 3)> with grad <GPUBuffer with shape (3, 3)>>,
 <SparseTensor <GPUBuffer with shape (9,)> with grad None>)

In [25]:
model.W.cpu().data==model2.W.cpu().data

AttributeError: 'Tensor' object has no attribute 'data'

In [ ]:
model.W.grad